In [ ]:
#@title #🛠️ 0. Prepare the environment
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null
!echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list
!sudo apt update && sudo apt install -y ngrok openssh-server

-------------

In [ ]:
#@title #🔐 1. Setup SSH
#Variable

#@markdown ## 👤 Username
#@markdown Set username for your SSH user
SSH_USERNAME = "debuguser"  #@param {type:"string"}

#@markdown ---

#@markdown ## 🔑 Password
#@markdown Set password for your SSH user
SSH_PASSWORD = "password"  #@param {type:"string"}

#@markdown ---

#@markdown ## 🗝️ SSH Public Key (optional)
#@markdown Enter your SSH public key here to enable key-based login (more secure).
#@markdown If you leave this blank, you can still log in using your password.
SSH_PUBLIC_KEY = ""  #@param {type:"string"}

# Setup SSH
!sudo mkdir -p /var/run/sshd
!sudo useradd -m -s /bin/bash {SSH_USERNAME}
!echo "{SSH_USERNAME}:{SSH_PASSWORD}" | sudo chpasswd
!sudo mkdir -p /home/{SSH_USERNAME}/.ssh
!echo "{SSH_PUBLIC_KEY}" | sudo tee /home/{SSH_USERNAME}/.ssh/authorized_keys
!sudo chown -R {SSH_USERNAME}:{SSH_USERNAME} /home/{SSH_USERNAME}/.ssh
!sudo chmod 600 /home/{SSH_USERNAME}/.ssh/authorized_keys
!echo "{SSH_USERNAME} ALL=(ALL) NOPASSWD:ALL" | sudo tee /etc/sudoers.d/{SSH_USERNAME}
!sudo service ssh start

------------

In [ ]:
#@title #📂 2. Start miniserve (File sharing)
UPLOAD_FILE_ENABLED = True  #@param {type:"boolean"}
SHOW_HIDDEN_FILE = True     #@param {type:"boolean"}

!sudo wget -q -nc -O /usr/local/bin/miniserve https://github.com/svenstaro/miniserve/releases/download/v0.31.0/miniserve-0.31.0-x86_64-unknown-linux-gnu
!sudo chmod +x /usr/local/bin/miniserve

# Build miniserve command based on booleans
cmd = "/usr/local/bin/miniserve / --port 80 --directory-size"

if SHOW_HIDDEN_FILE:
    cmd += " --hidden"
if UPLOAD_FILE_ENABLED:
    cmd += " --upload-files"

# Start miniserve in background
import os
os.system(f"nohup sudo {cmd} &")


In [ ]:
#@title 🚀 3. Start ngrok

#@markdown ## 🔑 Ngrok Auth Token
#@markdown Enter your ngrok authentication token. Required to start ngrok tunnels.
NGROK_AUTH_TOKEN = ""  #@param {type:"string"}

#@markdown ---

#@markdown ## 🛠️ Ngrok API Key (Optional)
#@markdown Enter your ngrok API key for advanced features.
NGROK_API_KEY = ""  #@param {type:"string"}

#@markdown ---
NGROK_CONFIG_DIR = "/opt/ngrok"

import os
import time
import requests
import re

!mkdir -p {NGROK_CONFIG_DIR}

ngrok_config = f"""
version: 3

agent:
  authtoken: {NGROK_AUTH_TOKEN}
  api_key: {NGROK_API_KEY}

tunnels:
  miniserve:
    proto: http
    addr: 80
  ssh_server:
    addr: 22
    proto: tcp
"""

with open(NGROK_CONFIG_DIR + "/ngrok.yml", "w") as f:
    f.write(ngrok_config)

print(f"ngrok.yml config generated at " + NGROK_CONFIG_DIR + "/ngrok.yml")



!nohup ngrok start --all --config={NGROK_CONFIG_DIR}/ngrok.yml &> /dev/null &


time.sleep(5)
resp = requests.get("http://localhost:4040/api/tunnels").json()
tunnels = resp.get("tunnels", [])

miniserve_url = next((t['public_url'] for t in tunnels if t.get("proto") == "https"), None)
ssh_tunnel = next((t for t in tunnels if t['name'] == 'ssh_server'), None)

if ssh_tunnel:
    url = ssh_tunnel['public_url']
    match = re.match(r"tcp://([^:]+):(\d+)", url)
    if match:
        host, port = match.groups()
    else:
        host = port = None
else:
    url = host = port = None

print(f"🌐 Miniserve available at: {miniserve_url}")
print(f"🔑 Connect with: ssh debuguser@{host} -p {port}")
print(f"🔑 Connect with: sshpass -p '{SSH_PASSWORD}' ssh -o StrictHostKeyChecking=no debuguser@{host} -p {port}")
print(f"🔑 Password: {SSH_PASSWORD}")
print(f"TUNNEL: {url}")


--------

In [ ]:
#@title ⏳ 4. Keep runtime running
print("Runtime is running. Sleeping for 12 hours...")
print("Created by chunix64")
print("Updates available at: https://github.com/chunix64/jupyter-ssh")

import time

try:
    time.sleep(12 * 60 * 60)  # Sleep for 12 hours
except KeyboardInterrupt:
    print("\nStopped by user.")